### Transformata Hougha w przestrzeni ab - Domowe
Przestrzeń ρ,θ nie jest jedyną przestrzenią w której punkt odpowiada parametrom prostej. Np. można spróbować wykorzystać tradycyjne równanie prostej:

y=ax+b

W tej przestrzeni reprezentacją pęku prostych jest prosta. 

Zadanie: napisać funkcję, która jako argument przyjmuje obraz (binarny) oraz parametry:

- aMin - minimalna wartość parametru a
- aMax - maksymalna  wartość parametru a
- aSkok - skok parametru a
- bMin - minimalna wartość parametru b
- bMax  - maksymalna  wartość parametru b
- bSkok - skok parametru b
Jako wynik ma zwrócić macierz przestrzeni Hougha ab.

Uwagi:

- zadanie może wyglądać na skomplikowane ale tak naprawdę wymaga tylko starannego przemyślenia
- najważniejszy jest problem "adresowania" macierzy H. Można do tego wykorzystać dwa wektory A i B ze wszystkimi możliwymi wartościami jakie parametry a i b mogę przyjąć (w podanych zakresach z podanym skokiem). Wówczas indeksy tych wektorów będą współrzędnymi w macierzy H. Do stworzenia wektorów wykorzystaj funkcję _np.arange_
- na podstawie rozmiaru wektorów A i B (własność _shape_) należy stworzyć wyzerowaną macierz H (funkcja _np.zeros_) 
- dla każdego punktu krawędzi na obrazie należy:
    - używająć wektora A dla każdej z jego wartości a obliczyć odpowiednią wartość b - indeksy kolejnych wartości a (a_idx) będą jedną z współrzędnych macierzy H, natomiast drugą współrędną mozna uzyskać jako b_idx = np.argmin(abs(B - b)).
        - każde a_idx i b_idx wyznaczą punkt w macierzy H, który należy zinkrementować
- działanie funkcji należy przetestować na punktach generowanych w 'edytorku' z punktu "Transformacja Hougha dla małej liczby punktów". Proponowane parametry funkcji:
    - aMin: -5
    - aMax:  5
    - aSkok: 0.05 
    - bMin: -100
    - bMax:  100
    - bSkok: 1

Zastanów się w przypadku jakich prostych reprezentacja ab nie sprawdzi się.

In [ ]:
import matplotlib.pyplot as plt
import cv2
import numpy as np
from skimage.transform import hough_line, hough_line_peaks

In [ ]:
def hough_trans_ab(binary, a_min, a_max, a_step, b_min, b_max, b_step):
    b_values = np.arange(b_min, b_max, b_step)
    a_values = np.arange(a_min, a_max, a_step)
    
    H = np.zeros((b_values.shape[0], a_values.shape[0]))
    
    bin_x, bin_y = binary.shape
    for i in range(bin_x):
        for j in range(bin_y):
            if binary[i, j] == 1:
                for a_idx in range(len(a_values)):
                    b = i - a_values[a_idx] * j
                    b_idx = np.argmin(np.abs(b_values - b))
                    H[b_idx, a_idx] += 1
    return H

def show_hough(h, image):
    fig, axes = plt.subplots(1, 2, figsize=(15, 6))
    ax = axes.ravel()

    ax[0].imshow(image, 'gray')
    ax[0].set_title('Original Image')
    ax[0].set_axis_off()
    
    ax[1].imshow(h, 'gray')
    ax[1].set_title('Hough Transform in ab Space')
    
    plt.tight_layout()
    plt.show()

In [ ]:
a_min = -5
a_max = 5
a_step = 0.05
b_min = -100
b_max = 100
b_step = 1

# Przykład 1: pojedynczy punkt
im1 = np.zeros((100, 100))
im1[50, 50] = 1
hough_1 = hough_trans_ab(im1, a_min, a_max, a_step, b_min, b_max, b_step)
show_hough(hough_1, im1)

In [ ]:
# Przykład 2: trzy punkty
im2 = np.zeros((100, 100))
im2[80, 20] = 1
im2[40, 30] = 1
im2[60, 60] = 1
hough_3 = hough_trans_ab(im2, a_min, a_max, a_step, b_min, b_max, b_step)
show_hough(hough_3, im2)

In [ ]:
# Przykład 3: linia pionowa
im3 = np.zeros((100, 100))
im3[50:55, 50] = 1
hough_vertical = hough_trans_ab(im3, a_min, a_max, a_step, b_min, b_max, b_step)
show_hough(hough_vertical, im3)

In [ ]:
# Przykład 4: linia pozioma
im4 = np.zeros((100, 100))
im4[50, 50:55] = 1
hough_horizontal = hough_trans_ab(im4, a_min, a_max, a_step, b_min, b_max, b_step)
show_hough(hough_horizontal, im4)

Reprezentacja nie sprawdza się w przypadku prostych pionowych. dla linii pionowych, współczynnik nachylenia 𝑎 rośnie do nieskończoności. Oznacza to, że przy wyrażaniu linii pionowej za pomocą równania 𝑦=𝑎𝑥+𝑏, współczynnik 𝑎 staje się bardzo duży, a współczynnik 𝑏 staje się nieokreślony (bo linia jest równoległa do osi 𝑦). W praktyce oznacza to, że dla linii pionowych przestrzeń Hougha ab staje się "rozmyta". Wartości 𝑎 mogą rosnąć do nieskończoności, co sprawia, że trudno jest znaleźć jednoznaczne dopasowanie do linii pionowej w przestrzeni Hougha. Dlatego algorytmy wykorzystujące transformację Hougha ab mogą mieć trudności w identyfikacji linii pionowych lub mogą wymagać dodatkowych technik lub modyfikacji algorytmu w celu skutecznego ich wykrycia.